In [1]:
import pandas as pd
import numpy as np
from Code.UtilityFunctions.wikidata_functions import wikidata_query, category_query, min_qid, get_all_wikidata_claims, compare_qids, categories_dict_singular, get_qid_label
from Code.UtilityFunctions.get_data_path import get_path

In [2]:
location_mappings = pd.read_csv(get_path('location_mappings_expanded.csv'))

In [3]:
location_mappings

,coordinates,city,cityLabel,population,county,countyLabel,state,stateLabel,country,countryLabel
0,"-119.71,34.43",Q159288,Santa Barbara,88665.0,Q108106,Santa Barbara County,Q99,California,Q30,United States of America
1,"-90.34,38.55",Q966126,Grantwood Village,941.0,Q498034,St. Louis County,Q1581,Missouri,Q30,United States of America
2,"-110.88,32.22",Q18575,Tucson,542629.0,Q58688,Pima County,Q816,Arizona,Q30,United States of America
3,"-75.16,39.96",Q1345,Philadelphia,1603797.0,Q496900,Philadelphia County,Q1400,Pennsylvania,Q30,United States of America
4,"-75.47,40.34",Q1183476,Green Lane,490.0,Q378527,Montgomery County,Q1400,Pennsylvania,Q30,United States of America
...,...,...,...,...,...,...,...,...,...,...
11410,"-82.68,27.87",Q2151669,Pinellas Park,53093.0,Q494556,Pinellas County,Q812,Florida,Q30,United States of America
11411,"-119.8,39.32",Q7972623,Washoe Valley,3074.0,Q1227,Nevada,Q30,United States of America,Q30,United States of America
11412,"-90.05,30.02",Q34404,New Orleans,383997.0,Q486231,Orleans Parish,Q1588,Louisiana,Q30,United States of America
11413,"-90.23,38.66",Q38022,St. Louis,301578.0,Q1581,Missouri,Q30,United States of America,Q30,United States of America


In [4]:
location_mappings = location_mappings.dropna(subset=['city']).sort_values(by=['cityLabel'])

In [5]:
location_mappings.loc[location_mappings['cityLabel'].str.startswith('Q'), 'cityLabel'].unique()

array(['Q110916412', 'Q12530186', 'Q12536050', 'Q12564962', 'Q12564965',
       'Q20485552', 'Q20485660', 'Q31455252', 'Q31455253', 'Q31455254',
       'Quail Ridge', 'Quakertown'], dtype=object)

# Create triples for locations on business ids

In [6]:
biz = pd.read_json(get_path("yelp_academic_dataset_business.json"), lines=True)

In [7]:
biz

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."


In [8]:
biz2 = biz.copy()

In [9]:
biz2['long_lat_round'] = (biz2["longitude"].apply(round, args=(2,)).astype(str) + "," + biz2["latitude"].apply(round, args=(2,)).astype(str))

In [10]:
biz2

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,long_lat_round
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None,"-119.71,34.43"
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...","-90.34,38.55"
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...","-110.88,32.22"
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...","-75.16,39.96"
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...","-75.47,40.34"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3...","-113.49,53.47"
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3...","-86.77,36.12"
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None,"-86.07,39.91"
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ...","-89.95,38.78"


In [11]:
biz2 = biz2[['business_id','long_lat_round', 'address', 'city', 'state']]

In [12]:
biz_location_mapping_merge = biz2.merge(location_mappings, left_on='long_lat_round', right_on='coordinates', how='left')
biz_location_mapping_merge.rename(columns={'city_x':'city', 'state_x': 'state', 'city_y': 'city_qid', 'state_y': 'state_qid'}, inplace=True)

In [13]:
biz_location_mapping_merge

,business_id,long_lat_round,address,city,state,coordinates,city_qid,cityLabel,population,county,countyLabel,state_qid,stateLabel,country,countryLabel
0,Pns2l4eNsfO8kk83dixA6A,"-119.71,34.43","1616 Chapala St, Ste 2",Santa Barbara,CA,"-119.71,34.43",Q159288,Santa Barbara,88665.0,Q108106,Santa Barbara County,Q99,California,Q30,United States of America
1,mpf3x-BjTdTEA3yCZrAYPw,"-90.34,38.55",87 Grasso Plaza Shopping Center,Affton,MO,"-90.34,38.55",Q966126,Grantwood Village,941.0,Q498034,St. Louis County,Q1581,Missouri,Q30,United States of America
2,tUFrWirKiKi_TAnsVWINQQ,"-110.88,32.22",5255 E Broadway Blvd,Tucson,AZ,"-110.88,32.22",Q18575,Tucson,542629.0,Q58688,Pima County,Q816,Arizona,Q30,United States of America
3,MTSW4McQd7CbVtyjqoe9mw,"-75.16,39.96",935 Race St,Philadelphia,PA,"-75.16,39.96",Q1345,Philadelphia,1603797.0,Q496900,Philadelphia County,Q1400,Pennsylvania,Q30,United States of America
4,mWMc6_wTdE0EUBKIGXDVfA,"-75.47,40.34",101 Walnut St,Green Lane,PA,"-75.47,40.34",Q1183476,Green Lane,490.0,Q378527,Montgomery County,Q1400,Pennsylvania,Q30,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,"-113.49,53.47",3388 Gateway Blvd,Edmonton,AB,"-113.49,53.47",Q7622058,"Strathcona,",NaN,Q2096,Edmonton,Q1951,Alberta,Q16,Canada
150342,c8GjPIOTGVmIemT7j5_SyQ,"-86.77,36.12",2813 Bransford Ave,Nashville,TN,"-86.77,36.12",Q1960234,Berry Hill,2112.0,Q1177705,Davidson County,Q1509,Tennessee,Q30,United States of America
150343,_QAMST-NrQobXduilWEqSw,"-86.07,39.91","6020 E 82nd St, Ste 46",Indianapolis,IN,"-86.07,39.91",Q537853,Carmel,99757.0,Q506151,Hamilton County,Q1415,Indiana,Q30,United States of America
150344,mtGm22y5c2UHNXDFAjaPNw,"-89.95,38.78",2472 Troy Rd,Edwardsville,IL,"-89.95,38.78",Q577939,Edwardsville,26808.0,Q486448,Madison County,Q1204,Illinois,Q30,United States of America


In [27]:
index = biz_location_mapping_merge.loc[biz_location_mapping_merge['cityLabel'] != biz_location_mapping_merge['city']][['city','cityLabel']].drop_duplicates().index

biz_location_mapping_merge.loc[index]

,business_id,long_lat_round,address,city,state,coordinates,city_qid,cityLabel,population,county,countyLabel,state_qid,stateLabel,country,countryLabel
1,mpf3x-BjTdTEA3yCZrAYPw,"-90.34,38.55",87 Grasso Plaza Shopping Center,Affton,MO,"-90.34,38.55",Q966126,Grantwood Village,941.0,Q498034,St. Louis County,Q1581,Missouri,Q30,United States of America
6,n_0UpQx1hsNbnPUSlodU8w,"-90.34,38.63","8522 Eager Road, Dierbergs Brentwood Point",Brentwood,MO,"-90.34,38.63",Q967881,Richmond Heights,9286.0,Q498034,St. Louis County,Q1581,Missouri,Q30,United States of America
7,qkRM_2X51Yqxk3btlwAQIg,"-82.73,27.77",400 Pasadena Ave S,St. Petersburg,FL,"-82.73,27.77",Q4876370,Bear Creek,1906.0,Q494556,Pinellas County,Q812,Florida,Q30,United States of America
8,k0hlBqXX-Bt0vf1op7Jr1w,"-90.32,38.57",8025 Mackenzie Rd,Affton,MO,"-90.32,38.57",Q962845,Shrewsbury,6406.0,Q498034,St. Louis County,Q1581,Missouri,Q30,United States of America
11,eEOYSgkmpB90uNA7lDOMRA,"-82.46,27.96",,Tampa Bay,FL,"-82.46,27.96",Q49255,Tampa,384959.0,Q488874,Hillsborough County,Q812,Florida,Q30,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150054,2wpEbuFLi6OYr-i6RdtTxg,"-75.07,40.17",2044 E County Line Rd 190060000,UPPER MORELAND,PA,"-75.07,40.17",Q1184384,Warminster Heights,4010.0,Q1400,Pennsylvania,Q30,United States of America,Q30,United States of America
150094,7gfVwLaF3cJpgkYMbs85gg,"-75.12,40.14",1210 E Mermaid Ln,Glenside,PA,"-75.12,40.14",Q1182534,Willow Grove,13730.0,Q1400,Pennsylvania,Q30,United States of America,Q30,United States of America
150095,K8VgP7zH4RXusF3Nr3r-5A,"-82.67,28.32",11014 Little Rd,New Port Richey,FL,"-82.67,28.32",Q240417,Bayonet Point,26713.0,Q500992,Pasco County,Q812,Florida,Q30,United States of America
150183,_rM0QpGDdFk3Vux2kltQsQ,"-90.28,38.73",8009 West Florissant Ave,St.Louis,MO,"-90.28,38.73",Q961767,Country Club Hills,1014.0,Q498034,St. Louis County,Q1581,Missouri,Q30,United States of America


In [33]:
type(biz_location_mapping_merge['long_lat_round'][0])

str

In [34]:
biz_location_mapping_merge.loc[biz_location_mapping_merge['long_lat_round']=='-86.72,36.25']

,business_id,long_lat_round,address,city,state,coordinates,city_qid,cityLabel,population,county,countyLabel,state_qid,stateLabel,country,countryLabel
24,4iRzR7OaS-QaSXuvYxEGKA,"-86.72,36.25",1160 Gallatin Pike S,Nashville,TN,"-86.72,36.25",Q2273185,Goodlettsville,17789.0,Q1509,Tennessee,Q30,United States of America,Q30,United States of America
619,BFf44roOiEodYRu06Xq8cg,"-86.72,36.25",1215 Gallatin Pike S,Madison,TN,"-86.72,36.25",Q2273185,Goodlettsville,17789.0,Q1509,Tennessee,Q30,United States of America,Q30,United States of America
2666,QVbv447epxdOBOHK7loQPA,"-86.72,36.25",936 Gallatin Pike S,Madison,TN,"-86.72,36.25",Q2273185,Goodlettsville,17789.0,Q1509,Tennessee,Q30,United States of America,Q30,United States of America
11661,BaO8C8Zlo8kbOO2teNP8BQ,"-86.72,36.25",809 Gallatin Pike S,Madison,TN,"-86.72,36.25",Q2273185,Goodlettsville,17789.0,Q1509,Tennessee,Q30,United States of America,Q30,United States of America
15924,w5Q_dmk9r4AaS_DvPHxJow,"-86.72,36.25",903 Gallatin Pike S,Madison,TN,"-86.72,36.25",Q2273185,Goodlettsville,17789.0,Q1509,Tennessee,Q30,United States of America,Q30,United States of America
18450,vWAeYvrymJTbYJbFyzidmQ,"-86.72,36.25",102 East Palestine Ave,Madison,TN,"-86.72,36.25",Q2273185,Goodlettsville,17789.0,Q1509,Tennessee,Q30,United States of America,Q30,United States of America
18785,Uod2YZ8nLf5jVmvImJeVVw,"-86.72,36.25",907 Gallatin Pike S,Madison,TN,"-86.72,36.25",Q2273185,Goodlettsville,17789.0,Q1509,Tennessee,Q30,United States of America,Q30,United States of America
22262,gB1PKRyyBj0ZnKrb8byMCA,"-86.72,36.25",1202 Coreland Dr,Madison,TN,"-86.72,36.25",Q2273185,Goodlettsville,17789.0,Q1509,Tennessee,Q30,United States of America,Q30,United States of America
26891,o9eS7GmIZ4u9DO7n2jrC8w,"-86.72,36.25",903 Gallatin Pike S,Madison,TN,"-86.72,36.25",Q2273185,Goodlettsville,17789.0,Q1509,Tennessee,Q30,United States of America,Q30,United States of America
34246,cgJEdln3Q_GP34iY7dOpGw,"-86.72,36.25",1013 Gallatin Pike S,Madison,TN,"-86.72,36.25",Q2273185,Goodlettsville,17789.0,Q1509,Tennessee,Q30,United States of America,Q30,United States of America


In [16]:
def city_population_query(city_qid: str):
    try:
        city_population_query = f"""
        SELECT DISTINCT ?population
        WHERE {{
            ?city p:P1082 ?statement .
            VALUES ?city {{wd:{city_qid}}}
            ?statement ps:P1082 ?population .
            ?statement pq:P585 ?date .
            FILTER NOT EXISTS {{
                ?city p:P1082/pq:P585 ?date2 .
                FILTER(?date2 > ?date)
	        }}
        }} 
        """
        a = wikidata_query(city_population_query)
        return int(a['population.value'][0])
    except:
        return (None)

In [17]:
def county_query(city_qid: str):
    try:
        county_query = f"""SELECT ?county ?countyLabel
        WHERE
        {{
        wd:{city_qid} wdt:P131 ?county .
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" }}}}"""
        a = wikidata_query(county_query)
        return (a['county.value'][0][31:], a['countyLabel.value'][0])
    except:
        return (None, None)


In [18]:

def state_query(county_qid: str):
    try:
        state_query = f"""SELECT ?state ?stateLabel
        WHERE
        {{
        wd:{county_qid} wdt:P131 ?state .
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" }}}}"""
        a = wikidata_query(state_query)
        return (a['state.value'][0][31:], a['stateLabel.value'][0])
    except:
        return (None, None)


In [19]:

def country_query(state_qid: str):
    try:
        country_query = f"""SELECT ?country ?countryLabel
        WHERE
        {{
        wd:{state_qid} wdt:P17 ?country .
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" }}}}"""
        a = wikidata_query(country_query)
        return (a['country.value'][0][31:], a['countryLabel.value'][0])
    except:
        return (None, None)


In [36]:
location_mappings_test = pd.read_csv(get_path('location_mappings.csv')).head(25)
location_mappings_test

,coordinates,city,cityLabel
0,"-119.71,34.43",Q159288,Santa Barbara
1,"-90.34,38.55",Q966126,Grantwood Village
2,"-110.88,32.22",Q18575,Tucson
3,"-75.16,39.96",Q1345,Philadelphia
4,"-75.47,40.34",Q1183476,Green Lane
5,"-87.06,36.27",Q616259,Ashland City
6,"-90.34,38.63",Q967881,Richmond Heights
7,"-82.73,27.77",Q4876370,Bear Creek
8,"-90.32,38.57",Q962845,Shrewsbury
9,"-86.77,36.21",Q23197,Nashville


In [50]:
pd.Series(county_query('Q991464'))[0]


'Q498276'

0    None
1    None
dtype: object

In [60]:

def expand_location_mappings(location_mappings: pd.DataFrame):
    list_of_us_states = list(wikidata_query(sparql_query="SELECT ?state WHERE{?state wdt:P31 wd:Q35657.}")['state.value'].apply(lambda x: x[31:]))
    
    location_mappings['population'] = location_mappings.apply(lambda x: city_population_query(x.city), axis=1)
    location_mappings[['county_qid', 'countyLabel']] = location_mappings.apply(lambda x: pd.Series(county_query(x.city)) if pd.Series(county_query(x.city))[0] not in list_of_us_states else pd.Series([None, None]), axis=1)
    location_mappings[['state_qid', 'stateLabel']] = location_mappings.apply(lambda x: pd.Series(state_query(x.county_qid)) if x.county_qid is not None else pd.Series(state_query(x.city)), axis=1)
    location_mappings[['country_qid', 'countryLabel']] = location_mappings.apply(lambda x: pd.Series(country_query(x.state_qid)), axis=1)
    return location_mappings
expand_location_mappings(location_mappings_test)

,coordinates,city,cityLabel,population,county_qid,countyLabel,state_qid,stateLabel,country_qid,countryLabel
0,"-119.71,34.43",Q159288,Santa Barbara,88665,Q108106,Santa Barbara County,Q99,California,Q30,United States of America
1,"-90.34,38.55",Q966126,Grantwood Village,941,Q498034,St. Louis County,Q1581,Missouri,Q30,United States of America
2,"-110.88,32.22",Q18575,Tucson,542629,Q58688,Pima County,Q816,Arizona,Q30,United States of America
3,"-75.16,39.96",Q1345,Philadelphia,1603797,Q496900,Philadelphia County,Q1400,Pennsylvania,Q30,United States of America
4,"-75.47,40.34",Q1183476,Green Lane,490,Q378527,Montgomery County,Q1400,Pennsylvania,Q30,United States of America
5,"-87.06,36.27",Q616259,Ashland City,5193,Q495072,Cheatham County,Q1509,Tennessee,Q30,United States of America
6,"-90.34,38.63",Q967881,Richmond Heights,9286,Q498034,St. Louis County,Q1581,Missouri,Q30,United States of America
7,"-82.73,27.77",Q4876370,Bear Creek,1906,Q494556,Pinellas County,Q812,Florida,Q30,United States of America
8,"-90.32,38.57",Q962845,Shrewsbury,6406,Q498034,St. Louis County,Q1581,Missouri,Q30,United States of America
9,"-86.77,36.21",Q23197,Nashville,684410,Q1177705,Davidson County,Q1509,Tennessee,Q30,United States of America


In [63]:
location_mappings = pd.read_csv(get_path('location_mappings.csv'))
location_mappings.rename(columns={'city': 'city_qid'}, inplace=True) #TEMPORARY DELETE WHEN

In [64]:
location_mappings

,coordinates,city_qid,cityLabel
0,"-119.71,34.43",Q159288,Santa Barbara
1,"-90.34,38.55",Q966126,Grantwood Village
2,"-110.88,32.22",Q18575,Tucson
3,"-75.16,39.96",Q1345,Philadelphia
4,"-75.47,40.34",Q1183476,Green Lane
...,...,...,...
11410,"-82.68,27.87",Q2151669,Pinellas Park
11411,"-119.8,39.32",Q7972623,Washoe Valley
11412,"-90.05,30.02",Q34404,New Orleans
11413,"-90.23,38.66",Q38022,St. Louis
